In [1]:
import pandas as pd
import os
import shutil

from github import Github


In [9]:
project_releases = pd.read_csv('volatile_projects_complete_links_limit10.csv')
project_releases = project_releases[project_releases['project_name'] == 'Okhttp']

In [10]:
project_releases.head(5)

,project_name,project_link,version_name,commit,timestamp
264,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.2,"Commit(sha=""b84627ef4bc43f2096c1b764720f9ff238...",30/1/2021 19:44
265,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.1,"Commit(sha=""79d2a8d192d007b8acb2423d3ab6bba3a4...",30/1/2021 18:36
266,Okhttp,https://github.com/square/okhttp,parent-4.10.0-RC1,"Commit(sha=""4fd1e8f99833eebdd2e99f3456322aa197...",7/10/2020 3:28
267,Okhttp,https://github.com/square/okhttp,parent-4.9.1,"Commit(sha=""63dcd95bfa2345bb3f3d4abc6b6dbf36cf...",30/1/2021 18:09
268,Okhttp,https://github.com/square/okhttp,parent-4.9.0,"Commit(sha=""cbeaf8f955fff9caa5652ccc6c1393ec8b...",11/9/2020 21:08


In [18]:
finished_project_arr = []
finished_project = open('finished_ck_projects.txt', 'r')
for line in finished_project:
    finished_project_arr.append(line.split('\n')[0])
finished_project.close()


#finished_project_arr = ['web3j-web3j', 'voxeolabs-moho']
#finished_project_arr = ['voxeolabs-moho']


for project_name in project_releases['project_name'].unique():
    max_value_rsf = 0
    if project_name not in finished_project_arr:
        current_project = pd.DataFrame(project_releases[project_releases['project_name'] == project_name])
        current_project['rank'] = current_project['version_name'].rank()
        current_project = current_project.sort_values(by=['rank'], ascending=False)

        
        ### Download dataset
        try:
            os.mkdir('raw_sourcecode/' + project_name)
        except:
            pass
        for row in current_project.iterrows():
            print('Processing: ' + project_name + '/' + project_name + '_' + row[1]['version_name'])
            command = 'git clone ' + row[1]['project_link'] +  ' raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']
            #print(command)
            #print(os.system(command))
            os.system(command)
            
            commit = row[1]['commit'].replace('Commit(sha="','')
            commit = commit.replace('")','')
            
            command = 'cd raw_sourcecode/' + project_name + '/' + project_name +'_' + row[1]['version_name']+'/' + ' & git checkout ' + commit
            print(command)
            #print(os.system(command))
            os.system(command)
            
            
            print('Currently processing: ' + project_name)
            command = 'java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']
            print(command)
            os.system(command)
            os.rename('class.csv', project_name + '_' + row[1]['version_name'] + '_class.csv')
            os.rename('field.csv',  project_name + '_' + row[1]['version_name'] + '_field.csv')
            os.rename('method.csv', project_name + '_' + row[1]['version_name'] + '_method.csv')
            
            shutil.move(project_name + '_' + row[1]['version_name'] + '_class.csv', 'ck_results_invi/' + project_name + '_' + row[1]['version_name'] + '_class.csv')
            shutil.move(project_name + '_' + row[1]['version_name'] + '_field.csv', 'ck_results_invi/' + project_name + '_' + row[1]['version_name'] + '_field.csv')
            shutil.move(project_name + '_' + row[1]['version_name'] + '_method.csv', 'ck_results_invi/' + project_name + '_' + row[1]['version_name'] + '_method.csv')

            command = 'sudo rm raw_sourcecode/' + project_name + '/' + project_name +'_' + row[1]['version_name']+ ' -r'
            #print(command)
            #print(os.system(command))
            os.system(command)
            
            print('Finish processing: ' + project_name + '/' + project_name + '_' + row[1]['version_name'])
        
        finished_project = open('finished_ck_projects.txt', 'a')
        finished_project.write(project_name + '\n')
        finished_project.close()

Processing: Okhttp/Okhttp_parent-5.0.0-alpha.2
cd raw_sourcecode/Okhttp/Okhttp_parent-5.0.0-alpha.2/ & git checkout b84627ef4bc43f2096c1b764720f9ff2387dca32
Currently processing: Okhttp
java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar raw_sourcecode/Okhttp/Okhttp_parent-5.0.0-alpha.2
Finish processing: Okhttp/Okhttp_parent-5.0.0-alpha.2
Processing: Okhttp/Okhttp_parent-5.0.0-alpha.1
cd raw_sourcecode/Okhttp/Okhttp_parent-5.0.0-alpha.1/ & git checkout 79d2a8d192d007b8acb2423d3ab6bba3a4f999e1
Currently processing: Okhttp
java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar raw_sourcecode/Okhttp/Okhttp_parent-5.0.0-alpha.1
Finish processing: Okhttp/Okhttp_parent-5.0.0-alpha.1
Processing: Okhttp/Okhttp_parent-4.9.1
cd raw_sourcecode/Okhttp/Okhttp_parent-4.9.1/ & git checkout 63dcd95bfa2345bb3f3d4abc6b6dbf36cfb08aaf
Currently processing: Okhttp
java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar raw_sourcecode/Okhttp/Okhttp_parent-4.9.1
Finish processing: Okhttp/Okhttp_parent-4.9.1


KeyboardInterrupt: 

In [33]:
from google.cloud import storage
client = storage.Client()
print("Client created using default project: {}".format(client.project))

# Replace the string below with a unique name for the new bucket
bucket_name = "software-remodularization-ck-metrics-results-individual-version"
bucket = client.get_bucket(bucket_name)

print("Bucket name: {}".format(bucket.name))
print("Bucket location: {}".format(bucket.location))
print("Bucket storage class: {}".format(bucket.storage_class))


finished_project_arr = ['web3j-web3j']

for project_name in project_releases['project_name'].unique():
    max_value_rsf = 0
    if project_name in finished_project_arr:
        current_project = pd.DataFrame(project_releases[project_releases['project_name'] == project_name])
        current_project['rank'] = current_project['version_name'].rank()
        current_project = current_project.sort_values(by=['rank'], ascending=False)

        
        for row in current_project.iterrows():
            
            print('Currently processing: ' + project_name)
            command = 'java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']
            os.system(command)
            os.rename('class.csv', project_name + '_' + row[1]['version_name'] + '_class.csv')
            os.rename('field.csv',  project_name + '_' + row[1]['version_name'] + '_field.csv')
            os.rename('method.csv', project_name + '_' + row[1]['version_name'] + '_method.csv')
            
            shutil.move(project_name + '_' + row[1]['version_name'] + '_class.csv', 'ck_results_invi/' + project_name + '_' + row[1]['version_name'] + '_class.csv')
            shutil.move(project_name + '_' + row[1]['version_name'] + '_field.csv', 'ck_results_invi/' + project_name + '_' + row[1]['version_name'] + '_field.csv')
            shutil.move(project_name + '_' + row[1]['version_name'] + '_method.csv', 'ck_results_invi/' + project_name + '_' + row[1]['version_name'] + '_method.csv')

            #blob_name = project_name + '_class.csv'
            #blob = bucket.blob(blob_name)
            #source_file_name = project_name + '_class.csv'
            #blob.upload_from_filename(source_file_name)
            #print("File uploaded to {}.".format(bucket.name))

            #blob_name = project_name + '_field.csv'
            #blob = bucket.blob(blob_name)
            #source_file_name = project_name + '_field.csv'
            #blob.upload_from_filename(source_file_name)
            #print("File uploaded to {}.".format(bucket.name))

            #blob_name = project_name + '_method.csv'
            #blob = bucket.blob(blob_name)
            #source_file_name = project_name + '_method.csv'
            #blob.upload_from_filename(source_file_name)
            #print("File uploaded to {}.".format(bucket.name))

            #os.remove(project_name + '_' + row[1]['version_name'] + '_class.csv')
            #os.remove(project_name + '_' + row[1]['version_name'] + '_field.csv')
            #os.remove(project_name + '_' + row[1]['version_name'] + '_method.csv')

            #finished_project = open('finished_ck_projects.txt', 'a')
            #finished_project.write(project_name + '\n')
            #finished_project.close()

            print('Finish processing: ' + project_name + '/' + project_name + '_' + row[1]['version_name'])






Client created using default project: softwareremodularization
Bucket name: software-remodularization-ck-metrics-results-individual-version
Bucket location: US
Bucket storage class: STANDARD
Currently processing: web3j-web3j
Finish processing: web3j-web3j/web3j-web3j_v4.6.1
Currently processing: web3j-web3j
Finish processing: web3j-web3j/web3j-web3j_v4.6.0-android
Currently processing: web3j-web3j
Finish processing: web3j-web3j/web3j-web3j_v4.6.0
Currently processing: web3j-web3j
Finish processing: web3j-web3j/web3j-web3j_v4.5.9
Currently processing: web3j-web3j
Finish processing: web3j-web3j/web3j-web3j_v4.5.8
Currently processing: web3j-web3j
Finish processing: web3j-web3j/web3j-web3j_v4.5.7
Currently processing: web3j-web3j
Finish processing: web3j-web3j/web3j-web3j_v4.5.6
Currently processing: web3j-web3j
Finish processing: web3j-web3j/web3j-web3j_v4.5.5
Currently processing: web3j-web3j
Finish processing: web3j-web3j/web3j-web3j_v4.5.4
Currently processing: web3j-web3j
Finish proc

In [ ]:
for row in project_link.iterrows():
    project_name = row[1]['project_name']
    if project_name not in finished_project_arr:
        print('Currently processing: ' + project_name)
        command = 'java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar raw_sourcecode/' + project_name
        os.system(command)
        os.rename('class.csv', project_name + '_class.csv')
        os.rename('field.csv', project_name + '_field.csv')
        os.rename('method.csv', project_name + '_method.csv')

        blob_name = project_name + '_class.csv'
        blob = bucket.blob(blob_name)
        source_file_name = project_name + '_class.csv'
        blob.upload_from_filename(source_file_name)
        #print("File uploaded to {}.".format(bucket.name))

        blob_name = project_name + '_field.csv'
        blob = bucket.blob(blob_name)
        source_file_name = project_name + '_field.csv'
        blob.upload_from_filename(source_file_name)
        #print("File uploaded to {}.".format(bucket.name))

        blob_name = project_name + '_method.csv'
        blob = bucket.blob(blob_name)
        source_file_name = project_name + '_method.csv'
        blob.upload_from_filename(source_file_name)
        #print("File uploaded to {}.".format(bucket.name))

        os.remove(project_name + '_class.csv')
        os.remove(project_name + '_field.csv')
        os.remove(project_name + '_method.csv')

        finished_project = open('finished_ck_projects.txt', 'a')
        finished_project.write(project_name + '\n')
        finished_project.close()
        
        print('Finish processing: ' + project_name)
